In [4]:
import cv2
import mediapipe as mp
import os
import numpy as np
import csv

# Config
videos_dir = "dataset/"
output_csv = "landmarks.csv"  # Aquí se guarda el dataset
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)

with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['label'] + [f'{axis}{i}' for i in range(21) for axis in ['x', 'y', 'z']])

    for label in os.listdir(videos_dir):
        label_path = os.path.join(videos_dir, label)
        if not os.path.isdir(label_path): continue

        for video_name in os.listdir(label_path):
            video_path = os.path.join(label_path, video_name)
            cap = cv2.VideoCapture(video_path)

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret: break

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(image)

                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        landmarks = []
                        for lm in hand_landmarks.landmark:
                            landmarks.extend([lm.x, lm.y, lm.z])
                        writer.writerow([label] + landmarks)

            cap.release()
